In [4]:
import numpy as np
import time

def read_data(filename):
    input_f=open(filename, 'r')
    input_data=[]
    for line in input_f.readlines():
        input_data.append([float(val) for val in line.split(',')])
    input_data=np.array(input_data)
    train_x=input_data[:,:-2]
    train_y=input_data[:,-1]
    train_x=np.concatenate((train_x, np.ones((train_x.shape[0],1))),axis=1)
    n, dimension = train_x.shape
    return n, dimension, train_x, train_y.reshape(-1,1)
    
n, dimension, train_x, train_y = read_data('classification.txt')
# train_y = (train_y+1.0)/2
train_x, np.squeeze(train_y)

(array([[0.750072  , 0.97740794, 0.88565752, 1.        ],
        [0.87791369, 0.01925101, 0.50671112, 1.        ],
        [0.7773246 , 0.99406596, 0.82224385, 1.        ],
        ...,
        [0.5155064 , 0.15354364, 0.01275495, 1.        ],
        [0.2282263 , 0.97155357, 0.18305906, 1.        ],
        [0.36391513, 0.49207061, 0.71952659, 1.        ]]),
 array([ 1., -1.,  1., ...,  1.,  1.,  1.]))

In [5]:
pos_zero_threshold = 1e-7
neg_zero_threshold = -1e-7

def activate(val, threshold = neg_zero_threshold):
    activation_func = np.vectorize(lambda x: 1.0 if x > threshold else -1.0)
    return activation_func(val)

def predict(n, train_x, train_y, W):
#     output=[activation(np.sum(train_x[i,:]*W)*train_y[i,:]) for i in range(n)]
#     return np.array(output).reshape(-1,1)
    return activate(np.dot(train_x, W.reshape(-1,1))*train_y)

def get_accuracy(n, hypothesis, train_y):
    return (np.abs(hypothesis-train_y)<pos_zero_threshold).sum().astype('float')/n

def perceptron(n, dimension, train_x, train_y, max_epoch, learning_rate):
#     weight=np.ones((1,dimension))*-1.0
    weight = np.array([-1.0]*dimension)

    for epoch in range(max_epoch):
        hypothesis=predict(n, train_x, train_y, weight)#-train_y
#         print [activation(np.sum(train_x[i,:]*weight)*train_y[i,:]) for i in range(n)],'=?=', activation(np.dot(train_x, weight.reshape(-1,1))*train_y)

#         error = activation(np.dot(train_x, weight.reshape(-1,1))*train_y)

        accuracy = get_accuracy(n, hypothesis, train_y)
        
        delta = (hypothesis-train_y)*learning_rate*train_x
#         print 'Epoch #%d: accuracy_rate=%.2f'%(epoch+1,accuracy)#,(np.abs(error+train_y)<pos_zero_threshold).sum(),weight
        
        if (np.abs(hypothesis-train_y)<pos_zero_threshold).sum()==n:
            break

#         print delta.T, train_y, np.squeeze(np.dot(delta.T,train_y).T),weight
#         weight-=np.dot(delta.T,train_y).T
        weight-=np.squeeze(np.dot(delta.T,train_y).T)

    return weight, accuracy, hypothesis, epoch

start_time = time.time()
weight, accuracy, prediction, epoch = perceptron(n, dimension, train_x, train_y, max_epoch=5000, learning_rate=1e-6)
print 'After %d epoch(s), %.3f s elapsed:'% (epoch, time.time()-start_time)
print 'Weight matrix =', weight
print 'Accuracy rate=%.2f' % accuracy

After 2699 epoch(s), 0.949 s elapsed:
Weight matrix = [-0.1920512  -0.18236201 -0.19212661  0.54228   ]
Accuracy rate=1.00


In [6]:
# After 2541 epoch(s), 0.859 s elapsed:
# Weight matrix = [-0.00058477 -0.00057694 -0.00068078  0.00235   ]
# Accuracy rate=1.00

In [7]:
prediction=predict(n, train_x, train_y, weight)
print prediction.T, (prediction==train_y).sum()

[[ 1. -1.  1. ...  1.  1.  1.]] 2000


In [3]:
print (prediction).T, np.sign(prediction).T
print (np.sign(prediction)-train_y).T, ((np.sign(prediction)-train_y)<pos_zero_threshold).sum()

[[ 1. -1.  1. ...  1.  1.  1.]] [[ 1. -1.  1. ...  1.  1.  1.]]
[[0. 0. 0. ... 0. 0. 0.]] 2000


In [8]:
pos_zero_threshold = 1e-7
neg_zero_threshold = -1e-7

def sigmoid(val):
    if val>20:
        return 1
    if val<-1000:
        return 0
    return 1.0/(1.0+np.exp(-val))

def activate(val, threshold = neg_zero_threshold):
#     activation_func = np.vectorize(lambda x: 1.0 if x > threshold else -1.0)
    activation_func = np.vectorize(lambda x: sigmoid(x)*2.0-1.0)
#     activation_func = np.vectorize(lambda x: sigmoid(x))
    return activation_func(val)

def predict(n, train_x, train_y, W):
#     output=[activation(np.sum(train_x[i,:]*W)*train_y[i,:]) for i in range(n)]
#     return np.array(output).reshape(-1,1)
    return activate(np.dot(train_x, W.reshape(-1,1))*train_y)

def get_accuracy(n, hypothesis, train_y):
    return ((np.sign(prediction)-train_y)<pos_zero_threshold).sum().astype('float')/n

def logistic(n, dimension, train_x, train_y, max_epoch, learning_rate):
#     weight=np.ones((1,dimension))*-1.0
    weight = np.array([-1.0]*dimension)

    for epoch in range(max_epoch):
        hypothesis=predict(n, train_x, train_y, weight)#-train_y
        
#         print train_x.shape, train_y.shape, weight.shape
#         print [activation(np.sum(train_x[i,:]*weight)*train_y[i,:]) for i in range(n)],'=?=', activation(np.dot(train_x, weight.reshape(-1,1))*train_y)

#         error = activation(np.dot(train_x, weight.reshape(-1,1))*train_y)

        accuracy = get_accuracy(n, hypothesis, train_y)
        
#         print hypothesis.shape, train_x.shape
#         break
        delta = (hypothesis-train_y)*learning_rate*train_x/n
        print 'Epoch #%d: accuracy_rate=%.2f'%(epoch+1,accuracy),((np.sign(prediction)-train_y)<pos_zero_threshold).sum(),weight,hypothesis.T,delta
        
#         if ((np.sign(prediction)-train_y)<pos_zero_threshold).sum()==n:
#             break

#         print delta.T, train_y, np.squeeze(np.dot(delta.T,train_y).T),weight
#         weight-=np.dot(delta.T,train_y).T
        weight-=np.squeeze(np.dot(delta.T,train_y).T)

    return weight, accuracy, hypothesis, epoch

start_time = time.time()
weight, accuracy, prediction, epoch = logistic(n, dimension, train_x, train_y, max_epoch=1000, learning_rate=1e-4)
print 'After %d epoch(s), %.3f s elapsed:'% (epoch, time.time()-start_time)
print 'Weight matrix =', weight
print 'Accuracy rate=%.2f' % accuracy

Epoch #1: accuracy_rate=1.00 2000 [-1. -1. -1. -1.] [[-0.94748205  0.83424475 -0.94647541 ... -0.68628679 -0.83101858
  -0.85853774]] [[-7.30375874e-08 -9.51742211e-08 -8.62401061e-08 -9.73741023e-08]
 [ 8.05154293e-08  1.76555308e-09  4.64716111e-08  9.17122377e-08]
 [-7.56521614e-08 -9.67462478e-08 -8.00238715e-08 -9.73237706e-08]
 ...
 [-4.34645818e-08 -1.29459309e-08 -1.07542483e-09 -8.43143394e-08]
 [-2.08943296e-08 -8.89466322e-08 -1.67592274e-08 -9.15509291e-08]
 [-3.38175002e-08 -4.57265900e-08 -6.68633665e-08 -9.29268870e-08]]
Epoch #2: accuracy_rate=1.00 2000 [-0.99990799 -0.99990672 -0.99990797 -0.99981675] [[-0.94746031  0.83419725 -0.94645336 ... -0.68622167 -0.83097035
  -0.8584945 ]] [[-7.30367723e-08 -9.51731589e-08 -8.62391436e-08 -9.73730155e-08]
 [ 8.05133440e-08  1.76550735e-09  4.64704075e-08  9.17098625e-08]
 [-7.56513045e-08 -9.67451519e-08 -8.00229650e-08 -9.73226682e-08]
 ...
 [-4.34629033e-08 -1.29454309e-08 -1.07538330e-09 -8.43110833e-08]
 [-2.08937792e-08 -

Epoch #53: accuracy_rate=1.00 2000 [-0.99521839 -0.99515212 -0.99521713 -0.9904769 ] [[-0.9463408   0.83175954 -0.94531781 ... -0.68288799 -0.82849546
  -0.85627459]] [[-7.29947867e-08 -9.51184480e-08 -8.61895685e-08 -9.73170401e-08]
 [ 8.04063391e-08  1.76316093e-09  4.64086468e-08  9.15879770e-08]
 [-7.56071699e-08 -9.66887113e-08 -7.99762800e-08 -9.72658907e-08]
 ...
 [-4.33769767e-08 -1.29198377e-08 -1.07325726e-09 -8.41443994e-08]
 [-2.08655375e-08 -8.88240649e-08 -1.67361334e-08 -9.14247731e-08]
 [-3.37763204e-08 -4.56709084e-08 -6.67819465e-08 -9.28137294e-08]]
Epoch #54: accuracy_rate=1.00 2000 [-0.99512649 -0.99505895 -0.9951252  -0.99029389] [[-0.94631863  0.83171145 -0.94529533 ... -0.68282238 -0.82844664
  -0.85623077]] [[-7.29939553e-08 -9.51173647e-08 -8.61885868e-08 -9.73159317e-08]
 [ 8.04042281e-08  1.76311464e-09  4.64074284e-08  9.15855725e-08]
 [-7.56062960e-08 -9.66875938e-08 -7.99753557e-08 -9.72647665e-08]
 ...
 [-4.33752855e-08 -1.29193340e-08 -1.07321541e-09 -8

Epoch #105: accuracy_rate=1.00 2000 [-0.99044243 -0.99030998 -0.99043993 -0.98096629] [[-0.94517691  0.82924373 -0.94413738 ... -0.67946377 -0.82594149
  -0.85398098]] [[-7.29511367e-08 -9.50615684e-08 -8.61380282e-08 -9.72588457e-08]
 [ 8.02959057e-08  1.76073933e-09  4.63449072e-08  9.14621863e-08]
 [-7.55612908e-08 -9.66300397e-08 -7.99277497e-08 -9.72068689e-08]
 ...
 [-4.32887163e-08 -1.28935493e-08 -1.07107347e-09 -8.39731883e-08]
 [-2.08363933e-08 -8.86999988e-08 -1.67127570e-08 -9.12970744e-08]
 [-3.37345864e-08 -4.56144775e-08 -6.66994308e-08 -9.26990488e-08]]
Epoch #106: accuracy_rate=1.00 2000 [-0.99035065 -0.99021692 -0.99034811 -0.98078351] [[-0.94515431  0.82919504 -0.94411445 ... -0.67939767 -0.82589207
  -0.85393657]] [[-7.29502889e-08 -9.50604636e-08 -8.61370271e-08 -9.72577154e-08]
 [ 8.02937688e-08  1.76069248e-09  4.63436739e-08  9.14597522e-08]
 [-7.55603997e-08 -9.66289002e-08 -7.99268071e-08 -9.72057225e-08]
 ...
 [-4.32870125e-08 -1.28930418e-08 -1.07103131e-09 

Epoch #156: accuracy_rate=1.00 2000 [-0.98576389 -0.98556661 -0.98576019 -0.97165084] [[-0.94401306  0.82674639 -0.9429571  ... -0.67608076 -0.82340649
  -0.85170166]] [[-7.29074880e-08 -9.50046905e-08 -8.60864895e-08 -9.72006531e-08]
 [ 8.01862834e-08  1.75833552e-09  4.62816358e-08  9.13373195e-08]
 [-7.55154180e-08 -9.65713763e-08 -7.98792261e-08 -9.71478552e-08]
 ...
 [-4.32015182e-08 -1.28675773e-08 -1.06891597e-09 -8.38040380e-08]
 [-2.08074656e-08 -8.85768544e-08 -1.66895543e-08 -9.11703244e-08]
 [-3.36931126e-08 -4.55583983e-08 -6.66174294e-08 -9.25850830e-08]]
Epoch #157: accuracy_rate=1.00 2000 [-0.98567221 -0.98547366 -0.98566848 -0.9714683 ] [[-0.94399002  0.82669713 -0.94293374 ... -0.67601418 -0.82335648
  -0.85165667]] [[-7.29066238e-08 -9.50035643e-08 -8.60854690e-08 -9.71995009e-08]
 [ 8.01841209e-08  1.75828810e-09  4.62803877e-08  9.13348563e-08]
 [-7.55145098e-08 -9.65702148e-08 -7.98782654e-08 -9.71466868e-08]
 ...
 [-4.31998022e-08 -1.28670662e-08 -1.06887351e-09 

Epoch #208: accuracy_rate=1.00 2000 [-0.98099936 -0.98073606 -0.98099445 -0.96216539] [[-0.94280324  0.82416942 -0.94173034 ... -0.67260637 -0.82079087
  -0.84934703]] [[-7.28621153e-08 -9.49455659e-08 -8.60329150e-08 -9.71401619e-08]
 [ 8.00731655e-08  1.75585506e-09  4.62163468e-08  9.12084709e-08]
 [-7.54677385e-08 -9.65104023e-08 -7.98287913e-08 -9.70865172e-08]
 ...
 [-4.31119646e-08 -1.28409038e-08 -1.06670018e-09 -8.36303183e-08]
 [-2.07776180e-08 -8.84497939e-08 -1.66656137e-08 -9.10395436e-08]
 [-3.36502682e-08 -4.55004659e-08 -6.65327183e-08 -9.24673513e-08]]
Epoch #209: accuracy_rate=1.00 2000 [-0.98090779 -0.98064323 -0.98090286 -0.9619831 ] [[-0.94277974  0.82411956 -0.94170652 ... -0.6725393  -0.82074027
  -0.84930144]] [[-7.28612340e-08 -9.49444175e-08 -8.60318745e-08 -9.71389870e-08]
 [ 8.00709768e-08  1.75580706e-09  4.62150836e-08  9.12059778e-08]
 [-7.54668125e-08 -9.65092181e-08 -7.98278118e-08 -9.70853260e-08]
 ...
 [-4.31102360e-08 -1.28403889e-08 -1.06665741e-09 

...

Epoch #774: accuracy_rate=1.00 2000 [-0.92953605 -0.92856007 -0.92951941 -0.85978943] [[-0.92801762  0.79407178 -0.92674792 ... -0.6331766  -0.790258
  -0.82165693]] [[-7.23076014e-08 -9.42229869e-08 -8.53781653e-08 -9.64008810e-08]
 [ 7.87520092e-08  1.72688457e-09  4.54538065e-08  8.97035891e-08]
 [-7.48854283e-08 -9.57657265e-08 -7.92128311e-08 -9.63373962e-08]
 ...
 [-4.20956497e-08 -1.25381943e-08 -1.04155395e-09 -8.16588298e-08]
 [-2.04291978e-08 -8.69665777e-08 -1.63861477e-08 -8.95128999e-08]
 [-3.31464260e-08 -4.48191919e-08 -6.55365303e-08 -9.10828466e-08]]
Epoch #775: accuracy_rate=1.00 2000 [-0.9294458  -0.92846857 -0.92942914 -0.85961003] [[-0.92798871  0.79401523 -0.92671865 ... -0.63310436 -0.79020065
  -0.82160458]] [[-7.23065173e-08 -9.42215743e-08 -8.53768853e-08 -9.63994357e-08]
 [ 7.87495265e-08  1.72683013e-09  4.54523736e-08  8.97007613e-08]
 [-7.48842905e-08 -9.57642716e-08 -7.92116277e-08 -9.63359325e-08]
 ...
 [-4.20937878e-08 -1.25376397e-08 -1.04150788e-09 -8

Epoch #826: accuracy_rate=1.00 2000 [-0.92484643 -0.92380554 -0.92482884 -0.85046776] [[-0.92650077  0.79111474 -0.92521187 ... -0.6294087  -0.78725975
  -0.81891845]] [[-7.22507141e-08 -9.41488579e-08 -8.53109950e-08 -9.63250386e-08]
 [ 7.86222077e-08  1.72403826e-09  4.53788882e-08  8.95557370e-08]
 [-7.48257276e-08 -9.56893796e-08 -7.91496806e-08 -9.62605935e-08]
 ...
 [-4.19985309e-08 -1.25092674e-08 -1.03915099e-09 -8.14704350e-08]
 [-2.03949838e-08 -8.68209297e-08 -1.63587049e-08 -8.93629874e-08]
 [-3.30965973e-08 -4.47518156e-08 -6.54380098e-08 -9.09459226e-08]]
Epoch #827: accuracy_rate=1.00 2000 [-0.92475632 -0.92371418 -0.9247387  -0.85028864] [[-0.92647133  0.79105755 -0.92518205 ... -0.62933601 -0.78720177
  -0.81886546]] [[-7.22496098e-08 -9.41474188e-08 -8.53096910e-08 -9.63235663e-08]
 [ 7.86196974e-08  1.72398322e-09  4.53774393e-08  8.95528776e-08]
 [-7.48245687e-08 -9.56878975e-08 -7.91484547e-08 -9.62591026e-08]
 ...
 [-4.19966572e-08 -1.25087093e-08 -1.03910462e-09 

Epoch #878: accuracy_rate=1.00 2000 [-0.92016363 -0.91905794 -0.92014511 -0.84116088] [[-0.92495557  0.78812487 -0.92364728 ... -0.6256169  -0.78422849
  -0.81614644]] [[-7.21927635e-08 -9.40733433e-08 -8.52425689e-08 -9.62477785e-08]
 [ 7.84909653e-08  1.72116036e-09  4.53031381e-08  8.94062434e-08]
 [-7.47649178e-08 -9.56116142e-08 -7.90853567e-08 -9.61823638e-08]
 ...
 [-4.19007959e-08 -1.24801571e-08 -1.03673277e-09 -8.12808448e-08]
 [-2.03603931e-08 -8.66736781e-08 -1.63309599e-08 -8.92114244e-08]
 [-3.30461584e-08 -4.46836143e-08 -6.53382831e-08 -9.08073220e-08]]
Epoch #879: accuracy_rate=1.00 2000 [-0.92007365 -0.91896671 -0.9200551  -0.84098205] [[-0.92492557  0.78806705 -0.92361691 ... -0.62554374 -0.78416987
  -0.8160928 ]] [[-7.21916385e-08 -9.40718773e-08 -8.52412406e-08 -9.62462787e-08]
 [ 7.84884272e-08  1.72110471e-09  4.53016732e-08  8.94033524e-08]
 [-7.47637374e-08 -9.56101046e-08 -7.90841081e-08 -9.61808453e-08]
 ...
 [-4.18989104e-08 -1.24795955e-08 -1.03668612e-09 

Epoch #927: accuracy_rate=1.00 2000 [-0.91575731 -0.91459064 -0.91573792 -0.83240464] [[-0.92347321  0.78527736 -0.92214648 ... -0.62202211 -0.78134181
  -0.81350352]] [[-7.21371696e-08 -9.40008996e-08 -8.51769257e-08 -9.61736604e-08]
 [ 7.83659718e-08  1.71841949e-09  4.52309948e-08  8.92638678e-08]
 [-7.47065873e-08 -9.55370194e-08 -7.90236555e-08 -9.61073238e-08]
 ...
 [-4.18081391e-08 -1.24525592e-08 -1.03444021e-09 -8.11011054e-08]
 [-2.03274524e-08 -8.65334501e-08 -1.63045383e-08 -8.90670907e-08]
 [-3.29980685e-08 -4.46185892e-08 -6.52432005e-08 -9.06751760e-08]]
Epoch #928: accuracy_rate=1.00 2000 [-0.91566745 -0.91449954 -0.91564805 -0.83222608] [[-0.92344269  0.78521894 -0.92211558 ... -0.62194853 -0.7812826
  -0.81344927]] [[-7.21360249e-08 -9.39994081e-08 -8.51755741e-08 -9.61721344e-08]
 [ 7.83634075e-08  1.71836326e-09  4.52295148e-08  8.92609470e-08]
 [-7.47053864e-08 -9.55354836e-08 -7.90223852e-08 -9.61057788e-08]
 ...
 [-4.18062425e-08 -1.24519943e-08 -1.03439328e-09 -

Epoch #970: accuracy_rate=1.00 2000 [-0.91189563 -0.91067554 -0.91187551 -0.82473164] [[-0.92215106  0.78275437 -0.92080802 ... -0.61885023 -0.77878433
  -0.81115947]] [[-7.20875843e-08 -9.39362859e-08 -8.51183773e-08 -9.61075531e-08]
 [ 7.82552235e-08  1.71599098e-09  4.51670735e-08  8.91377184e-08]
 [-7.46545664e-08 -9.54704935e-08 -7.89686285e-08 -9.60404008e-08]
 ...
 [-4.17263829e-08 -1.24282081e-08 -1.03241735e-09 -8.09425114e-08]
 [-2.02982682e-08 -8.64092136e-08 -1.62811298e-08 -8.89392166e-08]
 [-3.29554167e-08 -4.45609172e-08 -6.51588701e-08 -9.05579735e-08]]
Epoch #971: accuracy_rate=1.00 2000 [-0.91180588 -0.91058455 -0.91178574 -0.82455332] [[-0.92212008  0.78269542 -0.92077665 ... -0.61877627 -0.77872459
  -0.81110468]] [[-7.20864222e-08 -9.39347715e-08 -8.51170051e-08 -9.61060038e-08]
 [ 7.82526361e-08  1.71593425e-09  4.51655801e-08  8.91347712e-08]
 [-7.46533473e-08 -9.54689345e-08 -7.89673389e-08 -9.60388324e-08]
 ...
 [-4.17244766e-08 -1.24276403e-08 -1.03237018e-09 